# Pokemon Showdown Gym - Google Colab Setup

This notebook sets up the Pokemon Showdown Gym environment for reinforcement learning in Google Colab.

## Features
- Automatic installation of all dependencies
- Pokemon Showdown server setup
- Environment testing and validation
- Ready-to-use training examples


## 1. Install Dependencies

First, let's install all the required packages:


In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y nodejs npm

# Install Python packages
!pip install -q poke-env==0.10.0
!pip install -q numpy==2.2.5
!pip install -q gymnasium
!pip install -q stable-baselines3
!pip install -q tensorboard
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q pandas

print("✅ All packages installed successfully!")


## 2. Clone and Setup Repository

Clone the showdown_gym repository and install it:


In [ ]:
# Clone the repository
!git clone https://github.com/UoA-CARES/showdown_gym.git

# Install the package in development mode
!cd showdown_gym && pip install -e .

print("✅ Repository cloned and installed!")


## 3. Setup Pokemon Showdown Server

Install and configure the Pokemon Showdown server:


In [ ]:
# Clone Pokemon Showdown server
!git clone https://github.com/smogon/pokemon-showdown.git

# Install dependencies
!cd pokemon-showdown && npm install

# Copy configuration
!cd pokemon-showdown && cp config/config-example.js config/config.js

print("✅ Pokemon Showdown server setup complete!")


## 4. Start Pokemon Showdown Server

Start the server in the background:


In [ ]:
import subprocess
import time
import os

# Start the server in the background
def start_showdown_server():
    os.chdir('/content/pokemon-showdown')
    process = subprocess.Popen(
        ['node', 'pokemon-showdown', 'start', '--no-security'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        preexec_fn=os.setsid
    )
    return process

# Start server
server_process = start_showdown_server()

# Wait a moment for server to start
time.sleep(5)

print("✅ Pokemon Showdown server started!")


## 5. Test Environment

Test that the environment is working correctly:


In [ ]:
import sys
sys.path.append('/content/showdown_gym')

from showdown_gym.showdown_environment import SingleShowdownWrapper
import numpy as np

# Test environment creation
try:
    env = SingleShowdownWrapper(team_type="random", opponent_type="random")
    print("✅ Environment created successfully!")
    
    # Test environment reset
    obs = env.reset()
    print(f"✅ Environment reset successful!")
    print(f"Observation shape: {obs.shape}")
    print(f"Action space size: {env.action_space.n}")
    
    # Test a random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    print(f"✅ Step successful! Reward: {reward}")
    
except Exception as e:
    print(f"❌ Error testing environment: {e}")
    print("Make sure the Pokemon Showdown server is running!")


## 6. Training Example

Here's a simple example of training an agent:


In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
import matplotlib.pyplot as plt

# Create vectorized environment
def make_env():
    return SingleShowdownWrapper(team_type="random", opponent_type="random")

env = make_vec_env(make_env, n_envs=1)

# Create DQN agent
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.001,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    target_update_interval=1000
)

print("✅ DQN model created successfully!")
print("You can now train the model with: model.learn(total_timesteps=10000)")
